In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
print(os.listdir("/home/sunilvikram/data"))

['mnt', 'sample_submission.csv.zip', 'test_supplement.csv.zip', '.ipynb_checkpoints', 'submission.csv', 'train_sample.csv.zip', 'Dalon_notebook.ipynb', 'submission_4c_withmany.csv', 'xgboost_8c.pkl', 'Untitled.ipynb', 'sample_submission.csv', 'BuildModel_merge_8c.ipynb', 'xgboost_4c_withmany.pkl', 'test.csv', 'train.csv.zip', 'test.csv.zip', 'BuildModel.ipynb']


In [3]:
usefulcolumns=['ip','app','device','os', 'channel', 'click_time', 'is_attributed']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        }

In [4]:
traindata= pd.read_csv('mnt/ssd/kaggle-talkingdata2/competition_files/train.csv',parse_dates=['click_time'],dtype=dtypes,usecols=usefulcolumns)

In [6]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 7 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
click_time       datetime64[ns]
is_attributed    uint8
dtypes: datetime64[ns](1), uint16(4), uint32(1), uint8(1)
memory usage: 3.6 GB


In [7]:
AttributedDF= traindata[traindata.is_attributed==1]

In [8]:
NotAttributed= traindata[traindata.is_attributed==0]

In [9]:
NotAttributed.shape

(184447044, 7)

In [10]:
NonAttrCount=AttributedDF.shape[0]*10 -AttributedDF.shape[0]

In [11]:
NonAttrCount= NonAttrCount *10

In [11]:
#lines=NotAttributed.shape[0]
#howmanytoskip = lines-NonAttrCount

#skiplines = np.random.choice(np.arange(1, lines), size=NonAttrCount, replace=False)

#sort the list
#skiplines=np.sort(skiplines)

In [12]:
NotAttributed=NotAttributed.sample(n=NonAttrCount)

In [13]:
NotAttributed.shape

(41116140, 7)

In [14]:
newdf = [AttributedDF,NotAttributed]

In [15]:
newtraindata= pd.concat(newdf)

In [16]:
newtraindata.shape

(41572986, 7)

In [17]:
import gc
gc.collect()

0

In [18]:
del traindata


In [19]:
del NotAttributed
del AttributedDF
gc.collect()

0

In [20]:
testdata= pd.read_csv('test.csv',parse_dates=['click_time'],dtype=dtypes)

In [22]:
newtraindata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41572986 entries, 103 to 8262032
Data columns (total 7 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
click_time       datetime64[ns]
is_attributed    uint8
dtypes: datetime64[ns](1), uint16(4), uint32(1), uint8(1)
memory usage: 1.1 GB


In [21]:
testdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18790469 entries, 0 to 18790468
Data columns (total 7 columns):
click_id      int64
ip            uint32
app           uint16
device        uint16
os            uint16
channel       uint16
click_time    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), uint16(4), uint32(1)
memory usage: 501.8 MB


In [24]:
Y = newtraindata['is_attributed']

In [25]:
newtraindata.drop(['is_attributed'], axis=1, inplace=True)

In [26]:
testdata.drop(['click_id'], axis=1, inplace=True)
gc.collect()

55

In [27]:
nrow_train = newtraindata.shape[0]
merge = pd.concat([newtraindata, testdata])

In [28]:
merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60363455 entries, 103 to 18790468
Data columns (total 6 columns):
ip            uint32
app           uint16
device        uint16
os            uint16
channel       uint16
click_time    datetime64[ns]
dtypes: datetime64[ns](1), uint16(4), uint32(1)
memory usage: 1.6 GB


In [29]:
def timeFeature(df):
    df['dayofclick'] = df['click_time'].dt.day.astype('uint8')
    df['hourofclick'] = df['click_time'].dt.hour.astype('uint8')
    df['minofclick'] = df['click_time'].dt.minute.astype('uint8')
    df['secofclick'] = df['click_time'].dt.second.astype('uint8')
    return df

In [30]:
merge=timeFeature(merge)
merge.head()

,ip,app,device,os,channel,click_time,dayofclick,hourofclick,minofclick,secofclick
103,204158,35,1,13,21,2017-11-06 15:41:07,6,15,41,7
1504,29692,9,1,22,215,2017-11-06 16:00:02,6,16,0,2
1798,64516,35,1,13,21,2017-11-06 16:00:02,6,16,0,2
2102,172429,35,1,46,274,2017-11-06 16:00:03,6,16,0,3
3056,199085,35,1,13,274,2017-11-06 16:00:04,6,16,0,4


In [21]:
import gc
gc.collect()

121

In [31]:
# Define all the groupby transformations
def groupByFeatures(df):
    GROUPBY_AGGREGATIONS = [
    
    # V1 - GroupBy Features #
    #########################    
    # Variance in day, for ip-app-channel
   # {'groupby': ['ip','app','channel'], 'select': 'dayofclick', 'agg': 'count'},
    # Variance in hour, for ip-app-os
    #{'groupby': ['ip','app','os'], 'select': 'hourofclick', 'agg': 'var'},
    # Variance in hour, for ip-day-channel
    #{'groupby': ['ip','dayofclick','channel'], 'select': 'hourofclick', 'agg': 'var'},
    # Count, for ip-day-hour
    {'groupby': ['ip','dayofclick','hourofclick'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app
    {'groupby': ['ip', 'app'], 'select': 'channel', 'agg': 'count'},        
    # Count, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app-day-hour
    {'groupby': ['ip','app','dayofclick','hourofclick'], 'select': 'channel', 'agg': 'count'},
    # Mean hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hourofclick', 'agg': 'mean'}, 
    
    # V2 - GroupBy Features #
    #########################
    # Average clicks on app by distinct users; is it an app they return to?
    {'groupby': ['app'], 
     'select': 'ip', 
     'agg': lambda x: float(len(x)) / len(x.unique()), 
     'agg_name': 'AvgViewPerDistinct'
    },
    # How popular is the app or channel?
    {'groupby': ['app'], 'select': 'channel', 'agg': 'count'},
    {'groupby': ['channel'], 'select': 'app', 'agg': 'count'}
    ]

# Apply all the groupby transformations
    for spec in GROUPBY_AGGREGATIONS:
    
    # Name of the aggregation we're applying
        agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']
    
    # Info
        print("Grouping by {}, and aggregating {} with {}".format(
        spec['groupby'], spec['select'], agg_name
        ))
    
    # Unique list of features to select
        all_features = list(set(spec['groupby'] + [spec['select']]))
    
    # Name of new feature
        new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])
    
    # Perform the groupby
        gp = df[all_features]. \
        groupby(spec['groupby'])[spec['select']]. \
        agg(spec['agg']). \
        reset_index(). \
        rename(index=str, columns={spec['select']: new_feature})
        
    # Merge back to X_train
        df=df.merge(gp, on=spec['groupby'], how='left')
    return df

In [32]:
merge=groupByFeatures(merge)

Grouping by ['ip', 'dayofclick', 'hourofclick'], and aggregating channel with count
Grouping by ['ip', 'app'], and aggregating channel with count
Grouping by ['ip', 'app', 'os'], and aggregating channel with count
Grouping by ['ip', 'app', 'dayofclick', 'hourofclick'], and aggregating channel with count
Grouping by ['ip', 'app', 'channel'], and aggregating hourofclick with mean
Grouping by ['app'], and aggregating ip with AvgViewPerDistinct
Grouping by ['app'], and aggregating channel with count
Grouping by ['channel'], and aggregating app with count


In [33]:
merge.head()

,ip,app,device,os,channel,click_time,dayofclick,hourofclick,minofclick,secofclick,ip_dayofclick_hourofclick_count_channel,ip_app_count_channel,ip_app_os_count_channel,ip_app_dayofclick_hourofclick_count_channel,ip_app_channel_mean_hourofclick,app_AvgViewPerDistinct_ip,app_count_channel,channel_count_app
0,204158,35,1,13,21,2017-11-06 15:41:07,6,15,41,7,1,1,1,1,15.0,2.311312,73472,122628
1,29692,9,1,22,215,2017-11-06 16:00:02,6,16,0,2,12,161,5,4,10.8,48.651072,6556705,579298
2,64516,35,1,13,21,2017-11-06 16:00:02,6,16,0,2,63,17,3,2,7.0,2.311312,73472,122628
3,172429,35,1,46,274,2017-11-06 16:00:03,6,16,0,3,3,2,1,1,16.0,2.311312,73472,30048
4,199085,35,1,13,274,2017-11-06 16:00:04,6,16,0,4,3,4,1,1,16.0,2.311312,73472,30048


In [34]:
import gc
gc.collect()

178

In [39]:
def timebtwClicks(df):
    GROUP_BY_NEXT_CLICKS = [
        {'groupby': ['ip']},
    {'groupby': ['ip', 'app']},
    {'groupby': ['ip', 'channel']},
    {'groupby': ['ip', 'os']},
    ]

# Calculate the time to next click for each group
    for spec in GROUP_BY_NEXT_CLICKS:
    
    # Name of new feature
        new_feature = '{}_nextClick'.format('_'.join(spec['groupby']))    
    
    # Unique list of features to select
        all_features = spec['groupby'] + ['click_time']
    
    # Run calculation
        print(">> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}")
        df[new_feature] = df[all_features].groupby(spec['groupby']).click_time.transform(lambda x: x.diff().shift(-1)).dt.second
    
    return df

In [40]:
merge = timebtwClicks(merge)

>> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}
>> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}
>> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}
>> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}


In [41]:
merge.head()

,ip,app,device,os,channel,click_time,dayofclick,hourofclick,minofclick,secofclick,...,ip_app_os_count_channel,ip_app_dayofclick_hourofclick_count_channel,ip_app_channel_mean_hourofclick,app_AvgViewPerDistinct_ip,app_count_channel,channel_count_app,ip_nextClick,ip_app_nextClick,ip_channel_nextClick,ip_os_nextClick
0,204158,35,1,13,21,2017-11-06 15:41:07,6,15,41,7,...,1,1,15.0,2.311312,73472,122628,23.0,NaN,NaN,8.0
1,29692,9,1,22,215,2017-11-06 16:00:02,6,16,0,2,...,5,4,10.8,48.651072,6556705,579298,4.0,40.0,6.0,47.0
2,64516,35,1,13,21,2017-11-06 16:00:02,6,16,0,2,...,3,2,7.0,2.311312,73472,122628,52.0,52.0,52.0,28.0
3,172429,35,1,46,274,2017-11-06 16:00:03,6,16,0,3,...,1,1,16.0,2.311312,73472,30048,25.0,25.0,NaN,50.0
4,199085,35,1,13,274,2017-11-06 16:00:04,6,16,0,4,...,1,1,16.0,2.311312,73472,30048,21.0,21.0,21.0,14.0


In [42]:
merge=merge.drop('click_time',axis=1)

In [43]:
merge.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'dayofclick', 'hourofclick',
       'minofclick', 'secofclick', 'ip_dayofclick_hourofclick_count_channel',
       'ip_app_count_channel', 'ip_app_os_count_channel',
       'ip_app_dayofclick_hourofclick_count_channel',
       'ip_app_channel_mean_hourofclick', 'app_AvgViewPerDistinct_ip',
       'app_count_channel', 'channel_count_app', 'ip_nextClick',
       'ip_app_nextClick', 'ip_channel_nextClick', 'ip_os_nextClick'],
      dtype='object')

In [44]:
train = merge[:nrow_train]
test = merge[nrow_train:]

In [46]:
train.shape

(41572986, 21)

In [47]:
test.shape

(18790469, 21)

In [48]:
import xgboost as xgb

# Split into X and y

X = train.as_matrix(columns=['ip', 'app', 'device', 'os', 'channel', 'dayofclick', 'hourofclick',
       'minofclick', 'secofclick', 'ip_dayofclick_hourofclick_count_channel',
       'ip_app_count_channel', 'ip_app_os_count_channel',
       'ip_app_dayofclick_hourofclick_count_channel',
       'ip_app_channel_mean_hourofclick', 'app_AvgViewPerDistinct_ip',
       'app_count_channel', 'channel_count_app', 'ip_nextClick',
       'ip_app_nextClick', 'ip_channel_nextClick', 'ip_os_nextClick'])

clf_xgBoost = xgb.XGBClassifier(
    max_depth = 4,
    subsample = 0.8,
    colsample_bytree = 0.7,
    colsample_bylevel = 0.7,
    scale_pos_weight = 9,
    min_child_weight = 0,
    reg_alpha = 4,
    n_jobs = 4, 
    objective = 'binary:logistic'
)
# Fit the models
clf_xgBoost.fit(X, Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
       colsample_bytree=0.7, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=0, missing=None, n_estimators=100,
       n_jobs=4, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=4, reg_lambda=1, scale_pos_weight=9, seed=None,
       silent=True, subsample=0.8)

In [49]:
del X
del train

In [50]:
gc.collect()

312

In [51]:
del Y

In [52]:
gc.collect()

0

In [53]:
import pickle

In [54]:
pickle.dump(clf_xgBoost, open('xgboost_4c_withmerge.pkl', 'wb'))

In [56]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18790469 entries, 41572986 to 60363454
Data columns (total 21 columns):
ip                                             uint32
app                                            uint16
device                                         uint16
os                                             uint16
channel                                        uint16
dayofclick                                     uint8
hourofclick                                    uint8
minofclick                                     uint8
secofclick                                     uint8
ip_dayofclick_hourofclick_count_channel        int64
ip_app_count_channel                           int64
ip_app_os_count_channel                        int64
ip_app_dayofclick_hourofclick_count_channel    int64
ip_app_channel_mean_hourofclick                float64
app_AvgViewPerDistinct_ip                      float64
app_count_channel                              int64
channel_count_app    

In [57]:
test.to_csv('test_withmerge.csv',index=False)

In [58]:
text_X = test.as_matrix(columns=['ip', 'app', 'device', 'os', 'channel', 'dayofclick', 'hourofclick',
       'minofclick', 'secofclick', 'ip_dayofclick_hourofclick_count_channel',
       'ip_app_count_channel', 'ip_app_os_count_channel',
       'ip_app_dayofclick_hourofclick_count_channel',
       'ip_app_channel_mean_hourofclick', 'app_AvgViewPerDistinct_ip',
       'app_count_channel', 'channel_count_app', 'ip_nextClick',
       'ip_app_nextClick', 'ip_channel_nextClick', 'ip_os_nextClick'])

In [59]:
pred=clf_xgBoost.predict_proba(text_X)

In [60]:
submission = pd.read_csv("sample_submission.csv")
submission['is_attributed'] = pred[:,1]
submission.to_csv("submission_4c_withmerge.csv", index=False)

In [61]:
from xgboost import plot_importance

In [75]:
plot_importance(clf_xgBoost)

ImportError: You must install matplotlib to plot importance

In [76]:
import matplotlib.pyplot as plt

ImportError: cannot import name '_Backend'

In [77]:
matplotlib.use('Agg')

/home/sunilvikram/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  """


In [73]:
import matplotlib.

'/home/sunilvikram/anaconda3/lib/python3.5/site-packages/matplotlib/mpl-data/matplotlibrc'